# API City prices for living cost from numbeo.com

In [ ]:
# import libraries
import requests
import os
import pandas as pd
import json
import sql_functions as sf

In [ ]:
# To be able to pull the API key from the .env, we need to import load_dotenv

from dotenv import load_dotenv
load_dotenv()

Info from API numbeo.com

yearLastUpdate': 2023

Our day_cost_bucket:
'item_id': 1, 'item_name': 'Meal, Inexpensive Restaurant, Restaurants'
'item_id': 3, 'item_name': 'McMeal at McDonalds (or Equivalent Combo Meal), Restaurants'
'item_id': 4, 'item_name': 'Domestic Beer (0.5 liter draught), Restaurants'
'item_id': 7, 'item_name': 'Water (0.33 liter bottle), Restaurants'
'item_id': 114, 'item_name': 'Cappuccino (regular), Restaurants'

'item_id': 13, 'item_name': 'Water (1.5 liter bottle), Markets'

'item_id': 18, 'item_name': 'One-way Ticket (Local Transport), Transportation'

'item_id': 108, 'item_name': 'Taxi 1km (Normal Tariff), Transportation'




Selection list of items from numbeo.com for day_cost_bucket:
'item_id': 1, 'item_name': 'Meal, Inexpensive Restaurant, Restaurants'
'item_id': 2, 'item_name': 'Meal for 2 People, Mid-range Restaurant, Three-course, Restaurants'
'item_id': 3, 'item_name': 'McMeal at McDonalds (or Equivalent Combo Meal), Restaurants'
'item_id': 4, 'item_name': 'Domestic Beer (0.5 liter draught), Restaurants'
'item_id': 6, 'item_name': 'Coke/Pepsi (0.33 liter bottle), Restaurants'
'item_id': 7, 'item_name': 'Water (0.33 liter bottle), Restaurants'
'item_id': 114, 'item_name': 'Cappuccino (regular), Restaurants'

'item_id': 13, 'item_name': 'Water (1.5 liter bottle), Markets'

'item_id': 18, 'item_name': 'One-way Ticket (Local Transport), Transportation'



'item_id': 108, 'item_name': 'Taxi 1km (Normal Tariff), Transportation'



In [ ]:
#Load city list from data/cities_numbeo excel file
city_list = pd.read_excel('data/Cities_numbeo.xlsx')


In [ ]:
city_list.head()

In [ ]:
#Split the list in three columns on comma ' , ' 
city_list[['city', 'country', 'country_2']] = city_list['City'].str.split(', ', expand=True)

In [ ]:
city_list.head()

In [ ]:
city_list.loc[city_list['country_2'].notnull(), 'country'] = city_list['country_2']

In [ ]:
city_list.head()

In [ ]:
# Check the result
city_list['country'].unique()

In [ ]:
# Drop the column City and country_2
city_list.drop(['City', 'country_2'], axis = 1, inplace = True)

In [ ]:
city_list.head()

In [ ]:
city = list(city_list['city'])

In [ ]:
city

In [ ]:
# Make a country list for API query
country = list(city_list['country'])

In [ ]:
country

In [ ]:
#Create a dictionary city, country for API query
city_count_dict = dict(zip(city, country))

In [ ]:
city_count_dict

In [ ]:
# #API download from www.numbeo.com 
url = 'https://www.numbeo.com/api/city_prices'
api_key = 'zlueew978sczoi'
#country_list ' don`t need, we take the values for countries from country_list in cell above 
desired_item_ids = [1, 3, 4, 7, 114, 13, 18, 108, 26]
currency = "EUR"

data_list = []

for city in city:
    # Send the HTTP GET request for each country with currency parameter
    response = requests.get(url, params={"api_key": api_key, "query": city, "currency": currency})
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Access the "prices" list from the response
        prices = data.get("prices", [])
        
        # Access the country's land information
        city = city
        
        # Initialize a list to store the desired items for the country
        desired_items = []
        
        # Iterate over the prices and check if the item_id matches the desired values
        for price in prices:
            item_id = price.get("item_id")
            if item_id in desired_item_ids:
                desired_items.append(price)
        
        # Add the desired items, land information, and currency to the data list
        for item in desired_items:
            item["city"] = city
            item["currency"] = currency
        data_list.extend(desired_items)
    else:
        print(f"Request for {city} failed with status code:", response.status_code)

# Create a DataFrame from the data list
df = pd.DataFrame(data_list)

# Print the DataFrame
df

In [ ]:
df.head(15)

In [ ]:
# #Drop column data_points, item_id, lowest_price and highest_price
# df_new = df.drop(["data_points", 'lowest_price', 'highest_price', 'currency', 'item_id'], axis=1, inplace=True)

In [ ]:
df_backup = df

In [ ]:
#Drop column data_points, item_id, lowest_price and highest_price
df_new = df.drop(["data_points", 'lowest_price', 'highest_price', 'currency'], axis=1)

In [ ]:
#Add the column country
df_new['country'] = df_new['city'].map(city_count_dict)

In [ ]:
df_new.head(14)

In [ ]:
df_new.info()

In [ ]:
# Calculate price_per_day and calc_price for rows with item_id = 26
df_new.loc[df_new['item_id'] == 26, 'price_per_day'] = df_new.loc[df_new['item_id'] == 26, 'average_price'] / 30
df_new.loc[df_new['item_id'] == 26, 'calc_price'] = df_new.loc[df_new['item_id'] == 26, 'price_per_day'] * 3.89

# For other rows where item_id is not 26, fill with NaN or any other default value
df_new.loc[df_new['item_id'] != 26, ['price_per_day', 'calc_price']] = None

# # Display the entire DataFrame without truncation
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

df_new.head(20)


In [80]:
# # Calculate price_per_day and calc_price for rows with item_id = 26
# df_new.loc[df_new['item_id'] == 26, 'average_price'] = df_new.loc[df_new['item_id'] == 26, 'average_price'] / 30
# df_new.loc[df_new['item_id'] == 26, 'calc_price'] = df_new.loc[df_new['item_id'] == 26, 'price_per_day'] * 3.89

# # For other rows where item_id is not 26, fill with NaN or any other default value
# df_new.loc[df_new['item_id'] != 26, ['price_per_day', 'calc_price']] = None

df_new_backup = df_new

df_new.loc[df_new['calc_price'].notnull(), 'average_price'] = df_new['calc_price']

df_new.head(20)

,item_id,average_price,item_name,city,country,price_per_day,calc_price
0,1,10.000000,"Meal, Inexpensive Restaurant, Restaurants",Aachen,Germany,NaN,NaN
1,3,9.000000,McMeal at McDonalds (or Equivalent Combo Meal)...,Aachen,Germany,NaN,NaN
2,4,3.500000,"Domestic Beer (0.5 liter draught), Restaurants",Aachen,Germany,NaN,NaN
3,7,2.225000,"Water (0.33 liter bottle) , Restaurants",Aachen,Germany,NaN,NaN
4,13,0.496250,"Water (1.5 liter bottle), Markets",Aachen,Germany,NaN,NaN
5,18,2.950000,"One-way Ticket (Local Transport), Transportation",Aachen,Germany,NaN,NaN
6,26,72.800630,"Apartment (1 bedroom) in City Centre, Rent Per...",Aachen,Germany,18.714815,72.800630
7,108,1.950000,"Taxi 1km (Normal Tariff), Transportation",Aachen,Germany,NaN,NaN
8,114,2.885714,"Cappuccino (regular), Restaurants",Aachen,Germany,NaN,NaN
9,1,17.457899,"Meal, Inexpensive Restaurant, Restaurants",Aalborg,Denmark,NaN,NaN


In [83]:
df_clean = df_new.drop(["price_per_day", 'calc_price'], axis=1)
df_clean.head(15)

,item_id,average_price,item_name,city,country
0,1,10.000000,"Meal, Inexpensive Restaurant, Restaurants",Aachen,Germany
1,3,9.000000,McMeal at McDonalds (or Equivalent Combo Meal)...,Aachen,Germany
2,4,3.500000,"Domestic Beer (0.5 liter draught), Restaurants",Aachen,Germany
3,7,2.225000,"Water (0.33 liter bottle) , Restaurants",Aachen,Germany
4,13,0.496250,"Water (1.5 liter bottle), Markets",Aachen,Germany
5,18,2.950000,"One-way Ticket (Local Transport), Transportation",Aachen,Germany
6,26,72.800630,"Apartment (1 bedroom) in City Centre, Rent Per...",Aachen,Germany
7,108,1.950000,"Taxi 1km (Normal Tariff), Transportation",Aachen,Germany
8,114,2.885714,"Cappuccino (regular), Restaurants",Aachen,Germany
9,1,17.457899,"Meal, Inexpensive Restaurant, Restaurants",Aalborg,Denmark


In [85]:
df_clean_backup = df_clean

df_clean.loc[df_clean['item_id'] == 108, 'tax_max'] = df_clean.loc[df_clean['item_id'] == 108, 'average_price'] * 3
df_clean.head(17)

,item_id,average_price,item_name,city,country,tax_max
0,1,10.000000,"Meal, Inexpensive Restaurant, Restaurants",Aachen,Germany,NaN
1,3,9.000000,McMeal at McDonalds (or Equivalent Combo Meal)...,Aachen,Germany,NaN
2,4,3.500000,"Domestic Beer (0.5 liter draught), Restaurants",Aachen,Germany,NaN
3,7,2.225000,"Water (0.33 liter bottle) , Restaurants",Aachen,Germany,NaN
4,13,0.496250,"Water (1.5 liter bottle), Markets",Aachen,Germany,NaN
5,18,2.950000,"One-way Ticket (Local Transport), Transportation",Aachen,Germany,NaN
6,26,72.800630,"Apartment (1 bedroom) in City Centre, Rent Per...",Aachen,Germany,NaN
7,108,1.950000,"Taxi 1km (Normal Tariff), Transportation",Aachen,Germany,5.850000
8,114,2.885714,"Cappuccino (regular), Restaurants",Aachen,Germany,NaN
9,1,17.457899,"Meal, Inexpensive Restaurant, Restaurants",Aalborg,Denmark,NaN


In [87]:
df_clean.loc[df_clean['tax_max'].notnull(), 'average_price'] = df_clean['tax_max']
df_clean.head(17)

,item_id,average_price,item_name,city,country,tax_max
0,1,10.000000,"Meal, Inexpensive Restaurant, Restaurants",Aachen,Germany,NaN
1,3,9.000000,McMeal at McDonalds (or Equivalent Combo Meal)...,Aachen,Germany,NaN
2,4,3.500000,"Domestic Beer (0.5 liter draught), Restaurants",Aachen,Germany,NaN
3,7,2.225000,"Water (0.33 liter bottle) , Restaurants",Aachen,Germany,NaN
4,13,0.496250,"Water (1.5 liter bottle), Markets",Aachen,Germany,NaN
5,18,2.950000,"One-way Ticket (Local Transport), Transportation",Aachen,Germany,NaN
6,26,72.800630,"Apartment (1 bedroom) in City Centre, Rent Per...",Aachen,Germany,NaN
7,108,5.850000,"Taxi 1km (Normal Tariff), Transportation",Aachen,Germany,5.850000
8,114,2.885714,"Cappuccino (regular), Restaurants",Aachen,Germany,NaN
9,1,17.457899,"Meal, Inexpensive Restaurant, Restaurants",Aalborg,Denmark,NaN


In [89]:
df_clean = df_clean.drop(['tax_max'], axis=1)
df_clean.head(17)

,item_id,average_price,item_name,city,country
0,1,10.000000,"Meal, Inexpensive Restaurant, Restaurants",Aachen,Germany
1,3,9.000000,McMeal at McDonalds (or Equivalent Combo Meal)...,Aachen,Germany
2,4,3.500000,"Domestic Beer (0.5 liter draught), Restaurants",Aachen,Germany
3,7,2.225000,"Water (0.33 liter bottle) , Restaurants",Aachen,Germany
4,13,0.496250,"Water (1.5 liter bottle), Markets",Aachen,Germany
5,18,2.950000,"One-way Ticket (Local Transport), Transportation",Aachen,Germany
6,26,72.800630,"Apartment (1 bedroom) in City Centre, Rent Per...",Aachen,Germany
7,108,5.850000,"Taxi 1km (Normal Tariff), Transportation",Aachen,Germany
8,114,2.885714,"Cappuccino (regular), Restaurants",Aachen,Germany
9,1,17.457899,"Meal, Inexpensive Restaurant, Restaurants",Aalborg,Denmark


In [91]:
# df_clean['item_name'].replace('1','3')
# df_clean.head(17)

df_clean_backup2 = df_clean

df_clean.loc[(df_clean['item_id'] == 108) & (df_clean['item_name'] == 'Taxi 1km (Normal Tariff), Transportation'), 'item_name'] = 'Taxi 3km (Normal Tariff), Transportation'
df_clean.head(21)


,item_id,average_price,item_name,city,country
0,1,10.000000,"Meal, Inexpensive Restaurant, Restaurants",Aachen,Germany
1,3,9.000000,McMeal at McDonalds (or Equivalent Combo Meal)...,Aachen,Germany
2,4,3.500000,"Domestic Beer (0.5 liter draught), Restaurants",Aachen,Germany
3,7,2.225000,"Water (0.33 liter bottle) , Restaurants",Aachen,Germany
4,13,0.496250,"Water (1.5 liter bottle), Markets",Aachen,Germany
5,18,2.950000,"One-way Ticket (Local Transport), Transportation",Aachen,Germany
6,26,72.800630,"Apartment (1 bedroom) in City Centre, Rent Per...",Aachen,Germany
7,108,5.850000,"Taxi 3km (Normal Tariff), Transportation",Aachen,Germany
8,114,2.885714,"Cappuccino (regular), Restaurants",Aachen,Germany
9,1,17.457899,"Meal, Inexpensive Restaurant, Restaurants",Aalborg,Denmark


In [ ]:
# Define the columns to be transposed
columns_to_transpose = ['item_name']

# Get the remaining columns
remaining_columns = df.columns.difference(columns_to_transpose)

# Transpose the DataFrame
transposed_df2 = pd.melt(df, id_vars=remaining_columns, value_vars=columns_to_transpose, var_name='prices', value_name='prices2')

# Display the resulting DataFrame
transposed_df2


## Push the DataFrame df to sql

In [ ]:
#Define schema for DBeaver
schema = 'capstone_travel_index'
engine = sf.get_engine()

In [ ]:
#Import dataset as table to DBeaver, rename df_clean to country_avg_price
table_name = 'city_living_prices'
if engine!=None:
    try:
        df.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

In [ ]:
#The city_living_prices table was imported successfully.